# "Tree cover Extent in X?" Widget 

Adds a country comparison ranked list to the current widget. This might be merged with the current [widget](https://goo.gl/gG5mWk) to make a double widget - or be a stand alone one... waiting to hear!

Data:

1. adm-0 (adm1 and 2 = None are hardcoded)
3. extent
3. gadm28 area

User Variables:

1. extent year
2. unit (ha/relative %)
3. locations (all)
4. canopy threshold


## Note for the Front-End

Here I have used a new query `all_extent` to retrieve the area and extent of each country as a list...
however, I suspect the same data can be scraped from the existing `extent` query in the front end by selecting adm0 as `None`.

In [3]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [4]:
# Variables
adm0 = 'BRA'
adm1 = None
adm2 = None

extent_year = 2000     # extent data 2000 or 2010
location = "All Region"
thresh = 30

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"


tags = ["summary", "land_cover"] #replaces current tree cover loss widget in summary

selectable_polynames = ['gadm28', 'primary_forest', 'ifl_2013', 'wdpa', 'primary_forest__wdpa',
                        'mining', 'ifl_2013__wdpa', 'primary_forest__mining', 'ifl_2013__mining',
                        'plantations', 'plantations__mining']

In [21]:
def all_extent(p_name, adm0, extent_year=2000, threshold=30):
    print(f'Request for extent')
    sql = (f"SELECT polyname, "
           f"SUM({extent_year}) as value, "
           f"SUM(area_gadm28) as total_area, "
           f"FROM data "
           f"WHERE polyname = '{p_name}' "
           f"AND thresh= {threshold} "
           f"GROUP BY polyname, iso")
    return sql


In [33]:
# Query for Extent and Area

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
extent_sql = all_extent(p_name=polynames[location], adm0=adm0, extent_year=extent_year_dict[extent_year], threshold=thresh)

properties = {"sql": extent_sql}
rl = requests.get(url, params = properties)
print(rl.url)
print(f'Status: {rl.status_code}')

extent_data = rl.json()['data']

#Example data structure
extent_data[0:5]

Request for extent
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+polyname%2C+SUM%28area_extent_2000%29+as+value%2C+SUM%28area_gadm28%29+as+total_area%2C+FROM+data+WHERE+polyname+%3D+%27gadm28%27+AND+thresh%3D+30+GROUP+BY+polyname%2C+iso
Status: 200


[{'iso': 'BRA',
  'polyname': 'gadm28',
  'total_area': 850033236.0111084,
  'value': 519187505.155631},
 {'iso': 'USA',
  'polyname': 'gadm28',
  'total_area': 931559466.0361328,
  'value': 279340822.8477372},
 {'iso': 'ROU',
  'polyname': 'gadm28',
  'total_area': 23833860.149253845,
  'value': 7915160.704204235},
 {'iso': 'RUS',
  'polyname': 'gadm28',
  'total_area': 1672238939.6393886,
  'value': 760475347.7894046},
 {'iso': 'MEX',
  'polyname': 'gadm28',
  'total_area': 195202320.32836914,
  'value': 53185271.23384352}]

# Ranking

Rank the countries according to extent (ha) or extent relative to size of country (%)

i.e. ```relative extent = 100 x (extent / gadm28_area)```

In [23]:
"""First clean the data and calculate the loss relative the extent in the user selected year.
Note: if no loss data, set to zero."""

cleaned_data = list(map(lambda x: {
    
    'iso': x.get('iso'),
    'extent': x.get('value',0),
    'rel_extent': 100*x.get('value',0)/x.get('total_area',1)
    }, extent_data))

cleaned_data[0:5]

[{'extent': 519187505.155631, 'iso': 'BRA', 'rel_extent': 61.07849471768727},
 {'extent': 279340822.8477372, 'iso': 'USA', 'rel_extent': 29.986365125605655},
 {'extent': 7915160.704204235, 'iso': 'ROU', 'rel_extent': 33.209730419820524},
 {'extent': 760475347.7894046, 'iso': 'RUS', 'rel_extent': 45.4764764629508},
 {'extent': 53185271.23384352, 'iso': 'MEX', 'rel_extent': 27.2462290122245}]

In [24]:
#Retrieve the country we're interested in
country_data = list(filter(lambda x: x.get('iso') == adm0, cleaned_data))[0]
country_data

{'extent': 519187505.155631, 'iso': 'BRA', 'rel_extent': 61.07849471768727}

### Sort by extent (ha)

In [25]:
loss_rank_list = sorted(cleaned_data, key=lambda k: k.get('extent'), reverse=True) 
loss_rank_list[0:3]

[{'extent': 760475347.7894046, 'iso': 'RUS', 'rel_extent': 45.4764764629508},
 {'extent': 519187505.155631, 'iso': 'BRA', 'rel_extent': 61.07849471768727},
 {'extent': 417979643.29624414, 'iso': 'CAN', 'rel_extent': 42.35883770031754}]

In [26]:
# Get country's rank and print adjacent values ('net_usd' and 'iso' in this case)

rank = 1
for i in loss_rank_list:
    if i.get('iso') == adm0:
        print('RANK =', rank)
        break
    else:
        rank += 1

if rank == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(loss_rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(loss_rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
loss_rank_list[rank-bottom_bound:rank+upper_bound]

RANK = 2


[{'extent': 760475347.7894046, 'iso': 'RUS', 'rel_extent': 45.4764764629508},
 {'extent': 519187505.155631, 'iso': 'BRA', 'rel_extent': 61.07849471768727},
 {'extent': 417979643.29624414, 'iso': 'CAN', 'rel_extent': 42.35883770031754},
 {'extent': 279340822.8477372, 'iso': 'USA', 'rel_extent': 29.986365125605655},
 {'extent': 199224308.03852844, 'iso': 'COD', 'rel_extent': 85.57011233842077}]

### Sort by Relative extent (%)

i.e. the extent relative to size of country

In [27]:
rel_rank_list = sorted(cleaned_data, key=lambda k: k.get('rel_extent'), reverse=True) 
rel_rank_list[0:3]

[{'extent': 2654594.891357422, 'iso': 'GNQ', 'rel_extent': 98.54297073194343},
 {'extent': 8162751.48828125, 'iso': 'GUF', 'rel_extent': 97.99109834278252},
 {'extent': 9384713.318237305, 'iso': 'LBR', 'rel_extent': 97.84091262153397}]

In [28]:
# Get country's rank and print adjacent values ('net_usd' and 'iso' in this case)

rank = 1
for i in rel_rank_list:
    if i.get('iso') == adm0:
        print('RANK =', rank)
        break
    else:
        rank += 1

if rank == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(rel_rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(rel_rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
rel_rank_list[rank-bottom_bound:rank+upper_bound]

RANK = 50


[{'extent': 56532371.68322315, 'iso': 'VEN', 'rel_extent': 61.981795343763785},
 {'extent': 27883713.37854004, 'iso': 'SWE', 'rel_extent': 61.95729882774149},
 {'extent': 519187505.155631, 'iso': 'BRA', 'rel_extent': 61.07849471768727},
 {'extent': 24296427.336868286, 'iso': 'PRY', 'rel_extent': 60.81288106060947},
 {'extent': 78067722.18915814, 'iso': 'PER', 'rel_extent': 60.64206722835451}]

# Dynamic Sentence

In [32]:
if location == 'All Region':
    location_phrase = 'region-wide'
else:
    location_phrase = 'in ' + location.lower()

print(f"As of {extent_year} {iso_to_countries[adm0]} had ",end="")
print(f"{country_data.get('extent')}ha of tree cover {location_phrase}, ",end="")
print(f"equivalent to {country_data.get('rel_extent')}% of the country.",end="")

As of 2000 Brazil had 519187505.155631ha of tree cover region-wide, equivalent to 61.07849471768727% of the country.